<a href="https://colab.research.google.com/github/manojmanivannan/ApacheSparkEssentials/blob/master/SparkStreamByPluralsight.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get update
!apt-get install -y openjdk-8-jdk-headless scala jq  ssh > /dev/null
!if ! [[ -f "spark-2.3.1-bin-hadoop2.7.tgz" ]]; then wget -q http://archive.apache.org/dist/spark/spark-2.3.1/spark-2.3.1-bin-hadoop2.7.tgz; tar xf spark-2.3.1-bin-hadoop2.7.tgz; fi
!if ! [[ -f "ngrok-stable-linux-amd64.zip" ]]; then wget -q https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip; unzip ngrok-stable-linux-amd64.zip; fi

!if ! [[ -f "goodls_linux_amd64" ]]; then wget -q https://github.com/tanaikech/goodls/releases/download/v2.0.1/goodls_linux_amd64; chmod +x goodls_linux_amd64; fi
!cat /etc/ssh/ssh_host_rsa_key.pub > /etc/ssh/authorized_keys
!apt-get install python3.7 python3-pip
!python3.7 -m pip install --upgrade pyspark findspark

Hit:1 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists... Done
Reading package lists... Done
Building dependency t

In [2]:
!export SPARK_HOME="/content/spark-2.3.1-bin-hadoop2.7"
!export PATH="$PATH:$SPARK_HOME/bin"

In [3]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"


In [4]:
!/content/spark-2.3.1-bin-hadoop2.7/sbin/stop-master.sh
!/content/spark-2.3.1-bin-hadoop2.7/sbin/stop-slave.sh
!/content/spark-2.3.1-bin-hadoop2.7/sbin/start-master.sh

stopping org.apache.spark.deploy.master.Master
stopping org.apache.spark.deploy.worker.Worker
starting org.apache.spark.deploy.master.Master, logging to /content/spark-2.3.1-bin-hadoop2.7/logs/spark--org.apache.spark.deploy.master.Master-1-8afaf94dcefa.out


In [5]:
!sleep 3 && grep -oP "MasterUI' on port ([0-9]+)" /content/spark-2.3.1-bin-hadoop2.7/logs/spark--org.apache.spark.deploy.master*.out 
!grep -oP "spark:.*" /content/spark-2.3.1-bin-hadoop2.7/logs/spark--org.apache.spark.deploy.master*.out 

MasterUI' on port 8081
spark://8afaf94dcefa:7077


In [6]:
!/content/spark-2.3.1-bin-hadoop2.7/sbin/start-slave.sh $(grep -oP "spark:.*" /content/spark-2.3.1-bin-hadoop2.7/logs/spark--org.apache.spark.deploy.master*.out)
!sleep 3 && grep -oP "WorkerUI' on port ([0-9]+)" /content/spark-2.3.1-bin-hadoop2.7/logs/spark--org.apache.spark.deploy.worker*.out 

starting org.apache.spark.deploy.worker.Worker, logging to /content/spark-2.3.1-bin-hadoop2.7/logs/spark--org.apache.spark.deploy.worker.Worker-1-8afaf94dcefa.out
WorkerUI' on port 8082


In [7]:
get_ipython().system_raw('./ngrok authtoken 2IeJHLKYs6eB68PKJQM1VkpPu00_5JXQQZW3hofy6Q6vqTX2')
from time import sleep
def ngrok_tunnel_by_port(port: int):
  get_ipython().system_raw('pkill -f ngrok')
  get_ipython().system_raw(f'./ngrok http {port} &')
  sleep(4)
  print(get_ipython().getoutput('curl -s http://localhost:4040/api/tunnels',split=True))



In [8]:
ngrok_tunnel_by_port(4041)

['{"tunnels":[{"name":"command_line (http)","uri":"/api/tunnels/command_line%20%28http%29","public_url":"http://2ee0-34-139-46-169.ngrok.io","proto":"http","config":{"addr":"http://localhost:4041","inspect":true},"metrics":{"conns":{"count":0,"gauge":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0},"http":{"count":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0}}},{"name":"command_line","uri":"/api/tunnels/command_line","public_url":"https://2ee0-34-139-46-169.ngrok.io","proto":"https","config":{"addr":"http://localhost:4041","inspect":true},"metrics":{"conns":{"count":0,"gauge":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0},"http":{"count":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0}}}],"uri":"/api/tunnels"}']


In [9]:
!jps

15728 Jps
15654 Worker
15566 Master


In [10]:
master_url = get_ipython().getoutput('grep -oP "spark:.*" /content/spark-2.3.1-bin-hadoop2.7/logs/spark--org.apache.spark.deploy.master*.out',split=True)
print('Master URL:',master_url[0])
# get_ipython().system_raw(f'nohup pyspark --master {master_url[0]} --num-executors 5 --driver-memory 4g --executor-memory 4g &' )

Master URL: spark://8afaf94dcefa:7077


In [11]:
!jps

15747 Jps
15654 Worker
15566 Master


In [12]:
# !export PYSPARK_PYTHON=/usr/bin/python3.7; export PYSPARK_DRIVER_PYTHON=/usr/bin/python3.7; /content/spark-2.3.1-bin-hadoop2.7/bin/pyspark --master 'spark://8afaf94dcefa:7077' --num-executors 5 --driver-memory 4g --executor-memory 4g

In [15]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf

# Spark session & context
conf = SparkConf().set("spark.master","spark://8afaf94dcefa:7077").set('spark.ui.port', '4050').set("spark.executor.instances", 4).set("spark.executor.cores", 4).set("spark.driver.memory",'4g').set("spark.executor.memory",'4g')
try:
  sc = SparkContext(conf=conf)
except ValueError:
  sc.stop()
  sc = SparkContext(conf=conf)

spark = SparkSession.builder.appName('PluralSight').getOrCreate()

NameError: ignored

In [ ]:
ngrok_tunnel_by_port(4050)

In [ ]:
spark

In [ ]:
happiness_df = spark.read.option("header","true").csv("2016.csv")